In [1]:
from collections import Counter
import os
from glob import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 500)
from datetime import datetime

## Functions to process and merge files

I imagine that i'll be doing the exercise of scraping data from [cars.com](www.cars.com) and analyzing it until I find something so I'm creating some simple functions to handle the collection of files and creation of a Dataframe.

In [2]:
def file_list(file_path, pattern):
    """Return a list of files from a directory"""
    list_of_files = glob(os.path.join(file_path, pattern))
    
    return list_of_files

In [3]:
def combine_data(files):
    """Take a list of files and return a DataFrame"""
    df_list = [pd.read_csv(file) for file in files]
    
    # Column to indicate which source file the data originated
    for dataframe, filename in zip(df_list, files):
        dataframe['source_file'] = filename.split('/')[-1]
    
    combined_data = pd.concat(df_list, ignore_index=True)
    
    return combined_data

## Create and preview the dataset

In [4]:
# Grab all of the raw .csv files that were scraped
FILE_PATH = '/Users/bdoucet/Documents/new-car/data'
PATTERN = '*.csv'

list_of_files = file_list(FILE_PATH, PATTERN)

In [5]:
# Combine into a single Dataframe
df = combine_data(list_of_files)

In [6]:
# Preview some basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      390 non-null    object
 1   web-scraper-start-url  390 non-null    object
 2   car_page               390 non-null    object
 3   car_page-href          390 non-null    object
 4   product_name           390 non-null    object
 5   price                  390 non-null    object
 6   mileage                390 non-null    object
 7   sold_by                389 non-null    object
 8   location               389 non-null    object
 9   rating                 374 non-null    object
 10  no_of_reviews          374 non-null    object
 11  fuel_type              390 non-null    object
 12  city_mpg               390 non-null    object
 13  highway_mpg            390 non-null    object
 14  drivetrain             390 non-null    object
 15  engine                 

Each record represents a unique car listing from [cars.com](www.cars.com). The listings in the dataset meet the following conditions:

* Cars within 50 miles of my zip code.
* The price of the car is less than $15,000.00.
* Two car makes (Honda and Toyota) across four models (Civic & Accord / Corolla & Camry).
* Only cars from dealerships (no private listings).


## Get the year, make, model, and trim for each car

The *product_name* field is a string that contains the information I want on the model year, make, and trim of each car. 

Just about every listing uses the following pattern: **2017 Honda Civic LX**

However, there are some that do not: **Certified 2017 Toyota Camry 4-DOOR LE SEDAN**

Normally I would try and split each element in the string, index certain elements, and then assign them to a new column. I'm going to use regular expressions to match on specific text and extract matches.

In [7]:
# Preview a sample of the 'product_name' field
df['product_name'][:5]

0         2016 Toyota Corolla L
1        2017 Toyota Corolla SE
2    2015 Toyota Corolla S Plus
3    2016 Toyota Corolla S Plus
4        2018 Toyota Corolla LE
Name: product_name, dtype: object

In [8]:
# Get the model year: match on 4 digit years
df['model_year'] = df['product_name'].str.extract(r'(\d{4})')

In [9]:
# Validate results for: year
df.model_year.value_counts()

2017    205
2016     50
2015     38
2014     31
2018     29
2013     20
2012      8
2011      4
2007      3
2010      1
2009      1
Name: model_year, dtype: int64

In [10]:
df.head()

,web-scraper-order,web-scraper-start-url,car_page,car_page-href,product_name,price,mileage,sold_by,location,rating,no_of_reviews,fuel_type,city_mpg,highway_mpg,drivetrain,engine,exterior_color,interior_color,stock,transmission,vin,contact_info,source_file,model_year
0,1592758351-489,https://www.cars.com/for-sale/searchresults.ac...,See all 42 photos\n ...,https://www.cars.com/vehicledetail/detail/8121...,2016 Toyota Corolla L,"$12,490","60,690 miles",Sold by The Honda Store,"Lancaster, MA 01523",(4.9),118 Reviews,Gasoline,28,37,FWD,4 Cyl - 1.8 L,Black,Black,F119294A,Variable,2T1BURHE4GC530490,(978) 627-0129,toyota-corolla.csv,2016
1,1592758456-518,https://www.cars.com/for-sale/searchresults.ac...,See all 24 photos\n ...,https://www.cars.com/vehicledetail/detail/8107...,2017 Toyota Corolla SE,"$13,790","35,480 miles",Sold by Bourne's Auto Center,"Easton, MA 02375",(4.9),3681 Reviews,Gasoline,28,35,FWD,1.8L I4 16V MPFI DOHC,Blue,Black / Gray,63308R,Automatic CVT,2T1BURHE8HC873546,(774) 250-3978,toyota-corolla.csv,2017
2,1592758362-492,https://www.cars.com/for-sale/searchresults.ac...,See all 25 photos\n ...,https://www.cars.com/vehicledetail/detail/8122...,2015 Toyota Corolla S Plus,"$12,292","47,877 miles",Sold by Mastria Volkswagen,"Raynham, MA 02767",(4.9),183 Reviews,Gasoline,28,37,FWD,1.8L I4 16V MPFI DOHC,Classic Silver Metallic,Black,VW9217B,Automatic,2T1BURHE8FC255978,(877) 448-4347,toyota-corolla.csv,2015
3,1592758234-453,https://www.cars.com/for-sale/searchresults.ac...,See all 29 photos\n ...,https://www.cars.com/vehicledetail/detail/8056...,2016 Toyota Corolla S Plus,"$13,497","39,550 miles",Sold by Jaffarian Toyota,"Haverhill, MA 01832",(4.8),2153 Reviews,Gasoline,29,37,FWD,1.8L I4 16V MPFI DOHC,Red,Black,15600,Automatic CVT,2T1BURHE1GC641370,(978) 971-3167,toyota-corolla.csv,2016
4,1592757983-384,https://www.cars.com/for-sale/searchresults.ac...,Used\n \n \n ...,https://www.cars.com/vehicledetail/detail/8137...,2018 Toyota Corolla LE,"$14,999","26,183 miles",Sold by Acura of Boston,"Boston, MA 02135",(4.7),349 Reviews,Gasoline,28,36,FWD,1.8L I4 16V MPFI DOHC,Black,Black,AL0827A,Automatic CVT,2T1BURHE8JC086571,(617) 415-5302,toyota-corolla.csv,2018


In [11]:
# Get the car make and model
df['make'] = df['product_name'].str.extract(r'(Toyota|Honda)')
df['model'] = df['product_name'].str.extract(r'(Accord|Civic|Camry|Corolla)')


In [12]:
df['make'].value_counts()

Toyota    246
Honda     144
Name: make, dtype: int64

In [13]:
df['model'].value_counts()

Corolla    163
Civic       93
Camry       83
Accord      51
Name: model, dtype: int64

In [14]:
# Preview the 3 new fields: year, make, and model
df[['product_name', 'model_year', 'make', 'model']].sample(5)

,product_name,model_year,make,model
386,2013 Toyota Camry SE,2013,Toyota,Camry
84,Certified 2017 Toyota Corolla LE,2017,Toyota,Corolla
241,2017 Honda Civic LX,2017,Honda,Civic
284,2017 Honda Accord LX,2017,Honda,Accord
234,2017 Honda Civic LX,2017,Honda,Civic


Getting the trim for each car is going to be more of a challenge and will involve more text processing. I'm going to take each product_name and split it into a list of words; then use the Counter function to see what the most commons words in each product listing are. Then i'm going to create a regular expression to extract everything that is not one of those words.

In [15]:
# Create a new field called 'product_name_text' and preview a sample
df['product_name_text'] = df['product_name'].str.lower().str.split()
df['product_name_text'][:5]

0          [2016, toyota, corolla, l]
1         [2017, toyota, corolla, se]
2    [2015, toyota, corolla, s, plus]
3    [2016, toyota, corolla, s, plus]
4         [2018, toyota, corolla, le]
Name: product_name_text, dtype: object

In [16]:
# Create a set object to store unique words
product_words = Counter()

In [17]:
df['product_name_text'].apply(product_words.update)

0      None
1      None
2      None
3      None
4      None
       ... 
385    None
386    None
387    None
388    None
389    None
Name: product_name_text, Length: 390, dtype: object

In [18]:
# Turn the Counter object into a DataFrame
product_words_df = pd.DataFrame.from_dict(data=product_words,
                                          orient='index')\
                                          .reset_index() \
                                          .rename(columns={'index': 'product_word',
                                                           0: 'count'})


In [19]:
product_words_df.sort_values(by='product_word', ascending=False)

,product_word,count
16,xse,5
12,xle,8
34,w/special,1
1,toyota,246
23,touring,1
25,sport,6
33,sedan,3
5,se,67
7,s,11
13,premium,3


By analyzing the DataFrame that has word counts I'm constructing a list of strings (trims) and passing that to a regular expression to match.

In [20]:
# List of trims to search for
trims = ['xse', 'xle', 'sport', 'se', 's', 'lx-p', 'lx', 'le', 'l', 'ex-l', 'ex']

In [21]:
# Convert the list into a regex, then use the str.extract method
df['trim'] = (df.product_name.str
                             .lower()
                             .str.findall('|'.join(trims))
                             .str[-1]
                             .str.upper())

In [22]:
# Preview the results
df[['product_name', 'trim']][:5]

,product_name,trim
0,2016 Toyota Corolla L,L
1,2017 Toyota Corolla SE,SE
2,2015 Toyota Corolla S Plus,S
3,2016 Toyota Corolla S Plus,S
4,2018 Toyota Corolla LE,LE


In [23]:
df[df.trim.isna()]

,web-scraper-order,web-scraper-start-url,car_page,car_page-href,product_name,price,mileage,sold_by,location,rating,no_of_reviews,fuel_type,city_mpg,highway_mpg,drivetrain,engine,exterior_color,interior_color,stock,transmission,vin,contact_info,source_file,model_year,make,model,product_name_text,trim
257,1592756649-265,https://www.cars.com/for-sale/searchresults.ac...,See all 5 photos\n ...,https://www.cars.com/vehicledetail/detail/8100...,2016 Honda Accord Touring,"$15,000","45,280 miles",NaN,NaN,NaN,NaN,Gasoline,21,34,Automatic,"45,280",Silver,Black,FWD,6-cylinder,NaN,NaN,honda-accord.csv,2016,Honda,Accord,"[2016, honda, accord, touring]",NaN
365,1592757021-290,https://www.cars.com/for-sale/searchresults.ac...,JUST ADDED\n \n ...,https://www.cars.com/vehicledetail/detail/8140...,2015 Toyota Camry,"$13,889","59,509 miles",Sold by Mastria Kia,"Raynham, MA 02767",(4.9),40 Reviews,Gasoline,K0206B,6-Speed Automatic,4T1BF1FKXFU025488,NaN,White,FWD,2.5L I4 16V MPFI DOHC,"59,509",NaN,(877) 441-8604,toyota-camry.csv,2015,Toyota,Camry,"[2015, toyota, camry]",NaN


In [24]:
df['mileage'] = df.mileage.str.replace(',', '').str.split().str[0]

In [25]:
df.head()

,web-scraper-order,web-scraper-start-url,car_page,car_page-href,product_name,price,mileage,sold_by,location,rating,no_of_reviews,fuel_type,city_mpg,highway_mpg,drivetrain,engine,exterior_color,interior_color,stock,transmission,vin,contact_info,source_file,model_year,make,model,product_name_text,trim
0,1592758351-489,https://www.cars.com/for-sale/searchresults.ac...,See all 42 photos\n ...,https://www.cars.com/vehicledetail/detail/8121...,2016 Toyota Corolla L,"$12,490",60690,Sold by The Honda Store,"Lancaster, MA 01523",(4.9),118 Reviews,Gasoline,28,37,FWD,4 Cyl - 1.8 L,Black,Black,F119294A,Variable,2T1BURHE4GC530490,(978) 627-0129,toyota-corolla.csv,2016,Toyota,Corolla,"[2016, toyota, corolla, l]",L
1,1592758456-518,https://www.cars.com/for-sale/searchresults.ac...,See all 24 photos\n ...,https://www.cars.com/vehicledetail/detail/8107...,2017 Toyota Corolla SE,"$13,790",35480,Sold by Bourne's Auto Center,"Easton, MA 02375",(4.9),3681 Reviews,Gasoline,28,35,FWD,1.8L I4 16V MPFI DOHC,Blue,Black / Gray,63308R,Automatic CVT,2T1BURHE8HC873546,(774) 250-3978,toyota-corolla.csv,2017,Toyota,Corolla,"[2017, toyota, corolla, se]",SE
2,1592758362-492,https://www.cars.com/for-sale/searchresults.ac...,See all 25 photos\n ...,https://www.cars.com/vehicledetail/detail/8122...,2015 Toyota Corolla S Plus,"$12,292",47877,Sold by Mastria Volkswagen,"Raynham, MA 02767",(4.9),183 Reviews,Gasoline,28,37,FWD,1.8L I4 16V MPFI DOHC,Classic Silver Metallic,Black,VW9217B,Automatic,2T1BURHE8FC255978,(877) 448-4347,toyota-corolla.csv,2015,Toyota,Corolla,"[2015, toyota, corolla, s, plus]",S
3,1592758234-453,https://www.cars.com/for-sale/searchresults.ac...,See all 29 photos\n ...,https://www.cars.com/vehicledetail/detail/8056...,2016 Toyota Corolla S Plus,"$13,497",39550,Sold by Jaffarian Toyota,"Haverhill, MA 01832",(4.8),2153 Reviews,Gasoline,29,37,FWD,1.8L I4 16V MPFI DOHC,Red,Black,15600,Automatic CVT,2T1BURHE1GC641370,(978) 971-3167,toyota-corolla.csv,2016,Toyota,Corolla,"[2016, toyota, corolla, s, plus]",S
4,1592757983-384,https://www.cars.com/for-sale/searchresults.ac...,Used\n \n \n ...,https://www.cars.com/vehicledetail/detail/8137...,2018 Toyota Corolla LE,"$14,999",26183,Sold by Acura of Boston,"Boston, MA 02135",(4.7),349 Reviews,Gasoline,28,36,FWD,1.8L I4 16V MPFI DOHC,Black,Black,AL0827A,Automatic CVT,2T1BURHE8JC086571,(617) 415-5302,toyota-corolla.csv,2018,Toyota,Corolla,"[2018, toyota, corolla, le]",LE


In [26]:
df['mileage'] = pd.to_numeric(df['mileage'], errors='coerce')

In [27]:
# Fixing the price
df['price'] = (df['price']
               .replace(to_replace=r'\$|,', value='', regex=True)
               .astype('int'))

In [28]:
df['price'].apply(type).value_counts()

<class 'int'>    390
Name: price, dtype: int64

In [29]:
df[['price', 'mileage']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   price    390 non-null    int64
 1   mileage  390 non-null    int64
dtypes: int64(2)
memory usage: 6.2 KB


In [30]:
df['seller'] = df['sold_by'].str[8:]

In [31]:
df[['sold_by', 'seller']].head()

,sold_by,seller
0,Sold by The Honda Store,The Honda Store
1,Sold by Bourne's Auto Center,Bourne's Auto Center
2,Sold by Mastria Volkswagen,Mastria Volkswagen
3,Sold by Jaffarian Toyota,Jaffarian Toyota
4,Sold by Acura of Boston,Acura of Boston


In [32]:
df['seller'] = (np.where(df['seller']
                        .str.contains('CarMax'), df.seller.str[:6], df.seller))

In [33]:
# Strip the UNIX timestamp
df['date_accessed'] = (df['web-scraper-order'].str
                      .split('-')
                      .str[0])

# Convert UNIX timestamp to datetime object
df['date_accessed'] = pd.to_datetime(df['date_accessed'],
                                        unit='s')

In [34]:
df.sample(5)

,web-scraper-order,web-scraper-start-url,car_page,car_page-href,product_name,price,mileage,sold_by,location,rating,no_of_reviews,fuel_type,city_mpg,highway_mpg,drivetrain,engine,exterior_color,interior_color,stock,transmission,vin,contact_info,source_file,model_year,make,model,product_name_text,trim,seller,date_accessed
219,1592755531-174,https://www.cars.com/for-sale/searchresults.ac...,PRICE DROP\n \n ...,https://www.cars.com/vehicledetail/detail/8136...,2013 Honda Civic EX,11632,44491,Sold by Prestige Auto Mart,"Westport, MA 02790",-4.6,555 Reviews,Gasoline,28,39,FWD,1.8L I4 16V MPFI SOHC,Crimson Pearl,Beige,W829,5-Speed Automatic,19XFB2F89DE225317,(855) 349-0943,honda-civic.csv,2013,Honda,Civic,"[2013, honda, civic, ex]",EX,Prestige Auto Mart,2020-06-21 16:05:31
373,1592757061-298,https://www.cars.com/for-sale/searchresults.ac...,See all 31 photos\n ...,https://www.cars.com/vehicledetail/detail/8130...,2015 Toyota Camry SE,12995,69999,Sold by South Easton Motor Sales,"South Easton, MA 02375",(5),320 Reviews,Gasoline,25,35,FWD,2.5L I4 16V MPFI DOHC,Silver,Black,17400,6-Speed Automatic,4T1BF1FK1FU089693,(774) 568-8002,toyota-camry.csv,2015,Toyota,Camry,"[2015, toyota, camry, se]",SE,South Easton Motor Sales,2020-06-21 16:31:01
157,1592758025-395,https://www.cars.com/for-sale/searchresults.ac...,See all 14 photos\n ...,https://www.cars.com/vehicledetail/detail/8137...,2016 Toyota Corolla LE,10995,64997,Sold by AGM Auto Sales,"Malden, MA 02148",NaN,NaN,Gasoline,29,38,FWD,1.8L I4 16V MPFI DOHC,Gray,Gray,591359,Automatic CVT,2T1BURHE3GC591359,(781) 873-7097,toyota-corolla.csv,2016,Toyota,Corolla,"[2016, toyota, corolla, le]",LE,AGM Auto Sales,2020-06-21 16:47:05
376,1592757167-316,https://www.cars.com/for-sale/searchresults.ac...,See all 30 photos\n ...,https://www.cars.com/vehicledetail/detail/8126...,2014 Toyota Camry LE,12995,47045,Sold by South Easton Motor Sales,"South Easton, MA 02375",(5),320 Reviews,Gasoline,25,35,FWD,2.5L I4 16V MPFI DOHC,Gray,Gray,17385,6-Speed Automatic,4T4BF1FK8ER377182,(774) 568-8002,toyota-camry.csv,2014,Toyota,Camry,"[2014, toyota, camry, le]",LE,South Easton Motor Sales,2020-06-21 16:32:47
350,1592757324-344,https://www.cars.com/for-sale/searchresults.ac...,Used\n \n \n ...,https://www.cars.com/vehicledetail/detail/8130...,2011 Toyota Camry LE,11490,67488,Sold by Wellesley Toyota,"Wellesley, MA 02481",(4.8),905 Reviews,Gasoline,22,33,FWD,2.5L I4 16V MPFI DOHC,Aloe Green Metallic,Bisque,2WE887,6-Speed Automatic,4T1BF3EK5BU733459,(781) 943-6161,toyota-camry.csv,2011,Toyota,Camry,"[2011, toyota, camry, le]",LE,Wellesley Toyota,2020-06-21 16:35:24


In [35]:
df['listing_id'] = (df['car_page-href']
                    .str
                    .split('/')
                    .str[5])

In [36]:
df['zip'] = df['location'].str[-5:]

In [37]:
df['city'] = (df['location'].str
             .title()
             .str
             .split(',')
             .str[0])

In [38]:
df['state'] = (df['location'].str
               .split(',')
               .str[1]
               .str
               .strip()
               .str[0:2])

In [39]:
df['seller_reviews'] = (df['no_of_reviews'].str
                       .split()
                       .str[0])

In [40]:
df['seller_reviews'] = pd.to_numeric(df['seller_reviews'], errors='coerce')

In [41]:
df[['listing_id', 'model_year', 'make', 'model', 'trim', 
    'mileage', 'price', 'seller', 'seller_reviews',
   'city', 'state', 'zip',
    'source_file', 'date_accessed']].sample(5)

,listing_id,model_year,make,model,trim,mileage,price,seller,seller_reviews,city,state,zip,source_file,date_accessed
289,813709157,2015,Honda,Accord,SPORT,53642,13962,Midway Automotive,1155.0,Abington,MA,02351,honda-accord.csv,2020-06-21 16:22:31
246,811144875,2017,Honda,Civic,LX,26573,14641,Lundgren Honda,1196.0,Auburn,MA,01501,honda-civic.csv,2020-06-21 16:05:35
316,812214762,2013,Toyota,Camry,XLE,56773,12790,Ira Toyota of Danvers,698.0,Danvers,MA,01923,toyota-camry.csv,2020-06-21 16:37:02
173,792357845,2016,Honda,Civic,LX,20924,13995,Christmas Motors,31.0,Maynard,MA,01754,honda-civic.csv,2020-06-21 16:06:57
67,810574652,2014,Toyota,Corolla,S,22104,13000,Acton Toyota of Littleton,2884.0,Littleton,MA,01460,toyota-corolla.csv,2020-06-21 16:51:53


In [42]:
df['rating'] = df['rating'].str.replace(r'[\(\)]', '')

In [56]:
# Create a category for mileage
mileage_categories = ['0 - 10,000', '10,000 - 20,0000', 
          '20,000 - 30,000', '30,000- 40,0000', 
          '40,000 - 50,000', '50,000 - 60,000',
         '60,000 - 70,000']

df['mileage_range'] = pd.cut(df['mileage'], 
                             bins=np.linspace(0, 70000, 8),
                            labels=mileage_categories)

In [58]:
df['mileage_range'].value_counts()

30,000- 40,0000     115
40,000 - 50,000      75
20,000 - 30,000      60
50,000 - 60,000      56
60,000 - 70,000      55
10,000 - 20,0000     24
0 - 10,000            5
Name: mileage_range, dtype: int64

In [71]:
cut_bins = [0, 10000, 12000, 14000, 15000]
cut_labels = ['$10,000 or less', '$10,000 - $12,000', '$12,000 - $14,000', '$14,000 - $15,000']
df['price_range'] = pd.cut(df['price'], bins=cut_bins, labels=cut_labels)

In [69]:
df.price_range.value_counts()

$14,000 - $15,000    206
$12,000 - $14,000    132
$10,000 - $12,000     42
Under $10,000         10
Name: price_range, dtype: int64

In [73]:
df[df.price_range.str.contains('10,000 or less')][:5]

,web-scraper-order,web-scraper-start-url,car_page,car_page-href,product_name,price,mileage,sold_by,location,rating,no_of_reviews,fuel_type,city_mpg,highway_mpg,drivetrain,...,source_file,model_year,make,model,product_name_text,trim,seller,date_accessed,listing_id,zip,city,state,seller_reviews,mileage_range,price_range
92,1592757998-389,https://www.cars.com/for-sale/searchresults.ac...,JUST ADDED\n \n ...,https://www.cars.com/vehicledetail/detail/8140...,2015 Toyota Corolla S Premium,9495,70000,Sold by American Eagle Auto Sales,"Marlboro, MA 01752",4.3,24 Reviews,Gasoline,29,37,FWD,...,toyota-corolla.csv,2015,Toyota,Corolla,"[2015, toyota, corolla, s, premium]",S,American Eagle Auto Sales,2020-06-21 16:46:38,814068042,01752,Marlboro,MA,24.0,"60,000 - 70,000","$10,000 or less"
124,1592757938-370,https://www.cars.com/for-sale/searchresults.ac...,PRICE DROP\n \n ...,https://www.cars.com/vehicledetail/detail/8069...,2014 Toyota Corolla L,9942,52248,Sold by McGee Toyota,"Hanover, MA 02339",4.7,1296 Reviews,Gasoline,28,37,FWD,...,toyota-corolla.csv,2014,Toyota,Corolla,"[2014, toyota, corolla, l]",L,McGee Toyota,2020-06-21 16:45:38,806911735,02339,Hanover,MA,1296.0,"50,000 - 60,000","$10,000 or less"
139,1592758038-399,https://www.cars.com/for-sale/searchresults.ac...,See all 26 photos\n ...,https://www.cars.com/vehicledetail/detail/8138...,2013 Toyota Corolla LE,10000,65415,Sold by North End Motors,"Canton, MA 02021",4.9,3241 Reviews,Gasoline,26,34,FWD,...,toyota-corolla.csv,2013,Toyota,Corolla,"[2013, toyota, corolla, le]",LE,North End Motors,2020-06-21 16:47:18,813832163,02021,Canton,MA,3241.0,"60,000 - 70,000","$10,000 or less"
182,1592755736-226,https://www.cars.com/for-sale/searchresults.ac...,See all 34 photos\n ...,https://www.cars.com/vehicledetail/detail/8010...,2013 Honda Civic LX,8395,55316,Sold by 777 Auto LLC,"Weymouth, MA 02190",NaN,18 Reviews,Gasoline,28,36,FWD,...,honda-civic.csv,2013,Honda,Civic,"[2013, honda, civic, lx]",LX,777 Auto LLC,2020-06-21 16:08:56,801078134,02190,Weymouth,MA,18.0,"50,000 - 60,000","$10,000 or less"
184,1592755733-225,https://www.cars.com/for-sale/searchresults.ac...,See all 31 photos\n ...,https://www.cars.com/vehicledetail/detail/8107...,2014 Honda Civic LX,9990,53835,Sold by Wakefield Auto Gallery,"Wakefield, MA 01880",NaN,80 Reviews,Gasoline,28,36,FWD,...,honda-civic.csv,2014,Honda,Civic,"[2014, honda, civic, lx]",LX,Wakefield Auto Gallery,2020-06-21 16:08:53,810748781,01880,Wakefield,MA,80.0,"50,000 - 60,000","$10,000 or less"


In [80]:
df.rename(columns={'car_page-href': 'listing_url'}, inplace=True)

In [82]:
clean = df[['listing_id', 'model_year', 'make', 'model', 'trim',
    'interior_color', 'exterior_color', 'city_mpg', 'highway_mpg',
    'mileage', 'price', 'price_range', 'mileage_range',
    'seller', 'seller_reviews','city', 'state', 'zip',
    'listing_url', 'date_accessed']]

In [83]:
clean.to_csv('car_research_06-27-2020.csv')